In [17]:
import pandas as pd
import os


In [18]:
# script consts
PHEN_START = 2
VALUE_AT_END_OF_PHENOTYPES = 'const'
PLINK_PATH = "/sci/nosnap/michall/roeizucker/plink2"
BATCH_LENGTH = 250
MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY = 200


In [19]:
# # input files
# PHEN_PATH = "/sci/nosnap/michall/roeizucker/Hypothyroidism_hackaton/ukbb_dataset_filtered.csv"
# GWAS_PATH = "/sci/nosnap/michall/roeizucker/Hypothyroidism_hackaton/GWAS_FOR_PRS"
# PHENOTYPES = ["E03_F_reduced","E03_M_reduced","E03_reduced",
#              "E04_F_reduced","E04_M_reduced","E04_reduced",
#              "E05_F_reduced","E05_M_reduced","E05_reduced",
#              "E06_F_reduced","E06_M_reduced","E06_reduced"
#              "E07_F_reduced","E07_M_reduced","E07_reduced"]
# RESULT_SUFFIX = "PHENO1.glm.logistic"
# COVARS = []
# SPLIT_TO_TRAIN_TEST = True
# CREATE_PHENOTYPE_FILES = False
# CREATE_COVAR_FILE = True
# TRAIN_RATIO = 0.8
# input files
PHEN_PATH = "/cs/labs/michall/roeizucker/IIH/updated_phenotypes/ukbb_dataset_updated_11.csv"
GWAS_PATH = "/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe"
COVAR_FILE = "/cs/labs/michall/roeizucker/IIH/GWAS3/covariates.txt"
PHENS_TO_RUN = ["H47.1","G93.2","both_phen"]
PHENOTYPES = PHENS_TO_RUN
RESULT_SUFFIX = "PHENO1.glm.logistic.hybrid"
COVARS = ["sex","year_of_birth","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8"]
SPLIT_TO_TRAIN_TEST = False
CREATE_PHENOTYPE_FILES = True
CREATE_COVAR_FILE = True
TRAIN_RATIO = 0.8
NUMBER_OF_THREADS = 30
MEMORY_AMOUNT = 24
HADASSA_PLINK = "hadassa"
REDUCED_PLINK = "reduced"
ALL_PLINK = "all"
HWE = 10**-50
WHOLE_GENOME_STANDART = "standart"
plink_format = REDUCED_PLINK

MAF = 1e-7


JUST_CREATE_SCRIPT = True

In [4]:
!mkdir {GWAS_PATH}


mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_coding_low_hwe’: File exists


In [5]:
# !head /sci/nosnap/michall/roeizucker/Alopecia_project/ukbb_dataset_updated.csv

In [6]:
def create_files(gwas_path,phen_path,phenotypes):
#     !mkdir {gwas_path}
    !mkdir {gwas_path}/results
    !mkdir {gwas_path}/phenotypes
#     !cp /cs/labs/michall/roeizucker/10krun/runs/0:11/GWAS_delete_me/covariates.txt {gwas_path}/covariates.txt
    !cd {gwas_path}
    dataset = pd.read_csv(phen_path)
    if not CREATE_PHENOTYPE_FILES:
        return
    for phenotype_col in phenotypes:
        print(phenotype_col)
        file_name = phenotype_col.lower().replace(' ', '_').replace('-', '_') + '.txt'
        # use eid as fid, because we take one representitive of each family
        values = dataset[['eid', 'eid', phenotype_col]].dropna()
        if SPLIT_TO_TRAIN_TEST:
            values_case = values[values[phenotype_col] == 1]
            values_control = values[values[phenotype_col]== 0] 
            values_case = values_case.sample(int(len(values_case) * TRAIN_RATIO))
            values_control = values_control.sample(int(len(values_control) * TRAIN_RATIO))
            values = pd.concat([values_case,values_control])
        values.to_csv(os.path.join(f"{gwas_path}/phenotypes", file_name), \
            header = False, index = False, sep = '\t')
    if len(COVARS) != 0:
        covariate_values = dataset[['eid', 'eid'] + [covariate_col for covariate_col in COVARS if covariate_col != 'const']].dropna()
        covariate_values.to_csv(os.path.join( GWAS_PATH,'covariates.txt'), header = False, index = False, sep = '\t')
    else:
        ! cp {COVAR_FILE} {gwas_path}/covariates.txt
        # An array is created with all the values needed for running GWAS.
# The array will be sorted in batches for better file accessing
def create_the_data_file(phenotype_cols,gwas_path,data_file_path):
    values = []
    last_counter = 0
    cur_phen_counter = 0
    counter = 0
    for phen in phenotype_cols:
        base_path =  gwas_path
        for j in range(1,23):
            if os.path.exists(os.path.join(base_path ,f"results/{phen.lower()}_chr{j}.PHENO1.glm.linear")) or \
            os.path.exists(os.path.join(base_path ,f"results/{phen.lower()}_chr{j}.PHENO1.glm.logistic")) or \
            os.path.exists(os.path.join(base_path ,f"results/{phen.lower()}_chr{j}.{RESULT_SUFFIX}")):
#                 print("a")
                continue
            
            # TODO: change location of plink files to be const
            if plink_format == HADASSA_PLINK:
                values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                               os.path.join(base_path, f"results/{phen.lower()}_chr{j}"),
                               f"/cs/snapless/michall/hadasak/improve_PRS/bed_files/chr{j}_imputed_snp_id_filtered"
                               ,os.path.join(base_path ,"covariates.txt"),j])
            elif plink_format == REDUCED_PLINK:
                values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                               os.path.join(base_path, f"results/{phen.lower()}_chr{j}"),
                               f"/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}"
                               ,os.path.join(base_path ,"covariates.txt"),j])
            elif plink_format == ALL_PLINK:
                values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                               os.path.join(base_path, f"results/{phen.lower()}_chr{j}"),
                               f"/cs/biobank-genetics/EGAD00010001226/001/ukb_snp_chr{j}_v2"
                               ,os.path.join(base_path ,"covariates.txt"),j])
            elif plink_format == WHOLE_GENOME_STANDART:
                values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                               os.path.join(base_path, f"results/{phen.lower()}_chr{j}"),
                               f"/sci/archive/michall/roeizucker/standart_genetics/chr{j}_filtered_mafe-6_hwee-5_geno_02"
                               ,os.path.join(base_path ,"covariates.txt"),j])
                
            counter+=1
        cur_phen_counter+=1
        if cur_phen_counter%BATCH_LENGTH == 0:
            values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
            last_counter = counter
    values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
    df = pd.DataFrame(values, columns=["unsorted_counter","phenotype_path","output_path","partial_chromosome_file_path", "covariates","chr"])
    df.to_csv(data_file_path)


def write_mediator_script(mediator_script_path,data_file_path,gwas_path):
    with open(mediator_script_path, "w") as mediator_file:
        mediator_file.write(f'''import pandas as pd
import sys
import os
import os.path
butch_num = int(sys.argv[1]) 
curr_task = int(sys.argv[2])
BATCH_LENGTH = {BATCH_LENGTH}
SKIP_POINT = {MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY}
PLINK_PATH = "{PLINK_PATH}"
plink_format = "{plink_format}"
data_file_path = "{data_file_path}"
NUMBER_OF_THREADS = {NUMBER_OF_THREADS}
MEMORY_AMOUNT = {MEMORY_AMOUNT}
df = pd.read_csv(data_file_path)
print(df.loc[butch_num * BATCH_LENGTH + curr_task])
val = df.loc[butch_num * BATCH_LENGTH + curr_task]
location = butch_num * BATCH_LENGTH + curr_task
if curr_task == SKIP_POINT and location < 30500:
    new_batch  = butch_num + 1
    if not os.path.isfile("{gwas_path}_" + str(new_batch) + "_flag"):
        os.system("sbatch --array=0-"+str(BATCH_LENGTH - 1)+" --mem="+str(MEMORY_AMOUNT + 1)+" -c{NUMBER_OF_THREADS}--time=3-0 --killable --requeue --wrap=\\"{os.path.join( gwas_path,"master.sh")} "+str(new_batch)+"\\"")
if location < 30500:
    if plink_format == "{ALL_PLINK}":
        print(PLINK_PATH + " --bed " + str(df.loc[location].partial_chromosome_file_path).replace("snp","cal") + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch9.fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --covar-variance-standardize --freq --threads {NUMBER_OF_THREADS} --maf {MAF} --hwe {HWE} --geno 0.1 --memory " + str(MEMORY_AMOUNT * 1000) ,flush=True)
        os.system(PLINK_PATH + " --bed " + str(df.loc[location].partial_chromosome_file_path).replace("snp","cal") + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch9.fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --covar-variance-standardize --freq --threads {NUMBER_OF_THREADS} --maf {MAF} --hwe {HWE} --geno 0.1 --memory " +str(MEMORY_AMOUNT * 1000))
    else:
        print(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --covar-variance-standardize --freq --threads {NUMBER_OF_THREADS} --maf {MAF} --hwe {HWE} --geno 0.1 --memory " + str(MEMORY_AMOUNT * 1000),flush=True)
        os.system(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --covar-variance-standardize --freq --threads {NUMBER_OF_THREADS} --maf {MAF} --hwe {HWE} --geno 0.1 --memory " + str(MEMORY_AMOUNT * 1000))
    print("done!")
''')

def write_master_script(master_script_path,mediator_script_path,gwas_path):
    with open(master_script_path, "w") as master_file:
        master_file.write(f'''
FILE={gwas_path}_$1_flag
if ! test "$FILE" 
then 
    touch {gwas_path}_$1_flag
fi
python3 {mediator_script_path} $1 $SLURM_ARRAY_TASK_ID
''')
    !chmod 744 {master_script_path}


In [7]:
# # data_file_path = os.path.join(GWAS_PATH, "data_file.csv")
# # create_the_data_file(PHENOTYPES,GWAS_PATH,data_file_path)
# var = "/cs/biobank-genetics/EGAD00010001226/001/ukb_snp_chr1_v2.bed"
# print(var)
# print(str(var).replace("snp","cal"))

In [8]:
# master_script_path = os.path.join(GWAS_PATH , "master.sh")
# print(f'''how to run:
# sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{master_script_path} 0"''')

In [9]:
# script body (will be transferred to another file)


# TODO: add verbous argument, and better explain the output

# set input 
# TODO: change so it is accepted as params
phen_path = PHEN_PATH
gwas_path = GWAS_PATH

# do stuff
phen_dataset = pd.read_csv(PHEN_PATH)
phenotype_cols = list(phen_dataset)
phenotype_cols = phenotype_cols[2:phenotype_cols.index(VALUE_AT_END_OF_PHENOTYPES)]

# we will use the phenotypes parameter only if it exists
if len (PHENOTYPES) > 0:
    phenotype_cols = PHENOTYPES
    
# TODO: chnage so names are consts
data_file_path = os.path.join(gwas_path, "data_file.csv")
mediator_script_path = os.path.join(gwas_path , "mediator.py")
master_script_path = os.path.join(gwas_path , "master.sh")

create_files(gwas_path,phen_path,phenotype_cols)
create_the_data_file(phenotype_cols,gwas_path,data_file_path)
write_mediator_script(mediator_script_path,data_file_path,gwas_path)
write_master_script(master_script_path,mediator_script_path,gwas_path)

# TODO: add output folder
print(f'''how to run:
sbatch --array=0-263 --mem={MEMORY_AMOUNT + 1}g -c{NUMBER_OF_THREADS} --time=5-0 --requeue --killable --wrap="{master_script_path} 0"''')

mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_coding_low_hwe/results’: File exists
mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_coding_low_hwe/phenotypes’: File exists
H47.1
G93.2
both_phen
how to run:
sbatch --array=0-263 --mem=25g -c30 --time=5-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_coding_low_hwe/master.sh 0"


In [10]:

    
# print(phen_dataset[PHENS_TO_RUN].describe())
for phen in PHENS_TO_RUN:
    print(phen)
    print("both",len(phen_dataset[phen_dataset[phen] == 1]))
    print("female",len(phen_dataset[(phen_dataset[phen] == 1) &(phen_dataset["sex"] == 0)  ]))
    print("male",len(phen_dataset[(phen_dataset[phen] == 1) &(phen_dataset["sex"] == 1)]))

print(F"covars ({len(COVARS)})",COVARS)

H47.1
both 91
female 53
male 38
G93.2
both 116
female 77
male 39
both_phen
both 189
female 115
male 74
covars (10) ['sex', 'year_of_birth', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8']


In [11]:
create_the_data_file(phenotype_cols,gwas_path,data_file_path)

In [12]:

sbatch --array=0-3 --mem=25g -c30 --time=5-0 --requeue --killable --wrap="/sci/archive/michall/roeizucker/IIH/GWAS_standart/master.sh 0"


SyntaxError: invalid decimal literal (3184066541.py, line 1)

In [ ]:
# master_script_path = priority_gwas_dir + "master.sh"

# (mediator_script_path,data_file_path,gwas_path)
master_script_path

In [ ]:
pd.read_csv(data_file_path)

In [ ]:
print(f'''how to run:
sbatch --array=0-263 --mem=12g -c10 --time=3-0ס --wrap="{master_script_path} 0"''')

## combines GWAS results

In [20]:
import glob, os
# path = "/cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS/results"
# os.chdir(path)
path = os.path.join(GWAS_PATH,"results")
# files = []
# for file in glob.glob("*.glm.*"):
#     files.append(file)
phens = PHENOTYPES
for phen in phens:
    dataframes = []
    for j in range(1,23):
#         change so logistic/liner is detrmined generically
        file_name = os.path.join(path,f"{phen.lower()}_chr{j}.{RESULT_SUFFIX}")
        try:
            df = pd.read_csv(file_name,sep="\t")
            dataframes.append(df)
        except pd.errors.EmptyDataError:
            print(file_name)
            continue
        except FileNotFoundError:
            print(file_name)
            continue
    if len(dataframes) == 0:
        continue
    master_df = pd.concat(dataframes)
    master_df.to_csv(f"{os.path.join(path,phen)}.csv",index=False)
    print(f"{os.path.join(path,phen)}.csv")

/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/H47.1.csv
/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/G93.2.csv
/cs/labs/michall/roeizucker/IIH/updated_phenotypes/GWAS_standart_low_hwe/results/both_phen.csv


### combine single phenotype

In [ ]:
path = os.path.join(GWAS_PATH,"results")
phen = "L63"
dataframes = []
for j in range(1,23):
#         change so logistic/liner is detrmined generically
    file_name = os.path.join(path,f"{phen.lower()}_chr{j}.{RESULT_SUFFIX}")
    try:
        df = pd.read_csv(file_name,sep="\t")
        dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(file_name)
    except FileNotFoundError:
        print(file_name)
if not len(dataframes) == 0:
    master_df = pd.concat(dataframes)
    master_df.to_csv(f"{os.path.join(path,phen)}.csv",index=False)
    print(f"{os.path.join(path,phen)}.csv")

In [ ]:
os.path.join(path,phen)

In [ ]:
#delete this
df = pd.read_csv("/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/updated_PAP_genes1.csv")
df = df[df["P"] < 5*10**-5]
df.to_csv("/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/updated_PAP_with_genes_e-5.csv")

In [ ]:
df = pd.read_csv("/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/updated_Both_with_genes_e-5.csv")
df["gene_symbol"].value_counts()

In [ ]:
/sci/archive/michall/roeizucker/IIH/GWAS_standart/results/*_genes_e-5.csv

# Run GWAS for specific failed cases

In [ ]:
import re

# gwas_path = GWAS_PATH

os.path.join(GWAS_PATH,"data_file.csv")


In [ ]:
FAILED_FILES =["/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/D12_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/F171_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/I209_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/I20_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/I25_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/I48_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K219_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K52_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K52_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K52_chr3.PHENO1.glm.logistic", 
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K635_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K63_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K63_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K63_chr3.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K80_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K80_chr15.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/K80_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/M139_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/M13_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/M13_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/N40_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_155.1_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/220:229/GWAS/results/phecode_155.1_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_159.4_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/240:249/GWAS/results/phecode_159.4_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/260:269/GWAS/results/phecode_172.2_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/260:269/GWAS/results/phecode_172.2_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/260:269/GWAS/results/phecode_172.2_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/260:269/GWAS/results/phecode_172.2_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_172.2_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_172.2_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_172.2_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_172.2_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_198.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/370:379/GWAS/results/phecode_198.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/410:419/GWAS/results/phecode_202.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_202.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/420:429/GWAS/results/phecode_204.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_204.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/450:459/GWAS/results/phecode_208.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_208.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/450:459/GWAS/results/phecode_208.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_208.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_210.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/450:459/GWAS/results/phecode_210.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_211.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/460:469/GWAS/results/phecode_211.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_214.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/470:479/GWAS/results/phecode_214.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_214.0_chr5.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/470:479/GWAS/results/phecode_214.0_chr5.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_215.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/470:479/GWAS/results/phecode_215.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/480:489/GWAS/results/phecode_216.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_216.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/620:629/GWAS/results/phecode_250.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/620:629/GWAS/results/phecode_250.2_chr5.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/530:539/GWAS/results/phecode_228.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_262.0_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_250.2_chr5.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_255.21_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_276.14_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_250.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_261.2_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_276.11_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_260.6_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_260.6_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_275.5_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_261.2_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_262.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_276.14_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_276.14_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_272.1_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_228.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/700:709/GWAS/results/phecode_255.21_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/780:789/GWAS/results/phecode_262.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/780:789/GWAS/results/phecode_262.0_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/870:879/GWAS/results/phecode_275.5_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/830:839/GWAS/results/phecode_272.1_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/770:779/GWAS/results/phecode_261.2_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/770:779/GWAS/results/phecode_261.2_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/880:889/GWAS/results/phecode_276.11_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/760:769/GWAS/results/phecode_260.6_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/760:769/GWAS/results/phecode_260.6_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/890:899/GWAS/results/phecode_276.14_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/890:899/GWAS/results/phecode_276.14_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/890:899/GWAS/results/phecode_276.14_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_287.3_chr12.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_292.3_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_292.3_chr12.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_290.1_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_292.3_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_290.1_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_290.1_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1150:1159/GWAS/results/phecode_292.3_chr12.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1150:1159/GWAS/results/phecode_292.3_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1150:1159/GWAS/results/phecode_292.3_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1110:1119/GWAS/results/phecode_290.1_chr10.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1110:1119/GWAS/results/phecode_290.1_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1110:1119/GWAS/results/phecode_290.1_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1060:1069/GWAS/results/phecode_287.3_chr12.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1180:1189/GWAS/results/phecode_296.1_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1180:1189/GWAS/results/phecode_296.1_chr13.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_296.1_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_296.1_chr13.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1420:1429/GWAS/results/phecode_344.0_chr13.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1420:1429/GWAS/results/phecode_344.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/1420:1429/GWAS/results/phecode_344.0_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2260:2269/GWAS/results/phecode_458.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2260:2269/GWAS/results/phecode_458.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2260:2269/GWAS/results/phecode_458.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2340:2349/GWAS/results/phecode_478.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/30:39/GWAS/results/phecode_41.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2600:2609/GWAS/results/phecode_528.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2600:2609/GWAS/results/phecode_528.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_458.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_344.0_chr13.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_458.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_497.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_458.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_441.0_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_441.0_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_41.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_433.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_433.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_528.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_528.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_478.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_344.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_427.12_chr3.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_344.0_chr7.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_427.41_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_427.41_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2140:2149/GWAS/results/phecode_441.0_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2140:2149/GWAS/results/phecode_441.0_chr8.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2390:2399/GWAS/results/phecode_497.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2040:2049/GWAS/results/phecode_427.12_chr3.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2130:2139/GWAS/results/phecode_433.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2130:2139/GWAS/results/phecode_433.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2050:2059/GWAS/results/phecode_427.41_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2050:2059/GWAS/results/phecode_427.41_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2920:2929/GWAS/results/phecode_578.1_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2920:2929/GWAS/results/phecode_578.1_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2920:2929/GWAS/results/phecode_578.1_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2990:2999/GWAS/results/phecode_587.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3030:3039/GWAS/results/phecode_593.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3030:3039/GWAS/results/phecode_593.0_chr3.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3030:3039/GWAS/results/phecode_593.0_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_559.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_578.1_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_559.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_593.0_chr3.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_556.1_chr16.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_593.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_530.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_530.7_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_530.7_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_578.1_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_578.1_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_593.0_chr14.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_587.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_613.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_618.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2760:2769/GWAS/results/phecode_559.0_chr9.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2760:2769/GWAS/results/phecode_559.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2660:2669/GWAS/results/phecode_530.7_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2660:2669/GWAS/results/phecode_530.7_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2630:2639/GWAS/results/phecode_530.0_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3240:3249/GWAS/results/phecode_618.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3180:3189/GWAS/results/phecode_613.0_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/2750:2759/GWAS/results/phecode_556.1_chr16.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3840:3849/GWAS/results/phecode_716.9_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3520:3529/GWAS/results/phecode_671.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3570:3579/GWAS/results/phecode_687.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr16.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3270:3279/GWAS/results/phecode_619.4_chr15.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3990:3999/GWAS/results/phecode_735.21_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_703.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_716.9_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_741.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_626.12_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_681.2_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_626.12_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr15.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_671.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_687.0_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_735.21_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr16.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_626.12_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_619.4_chr6.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4050:4059/GWAS/results/phecode_741.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3700:3709/GWAS/results/phecode_703.0_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3540:3549/GWAS/results/phecode_681.2_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3320:3329/GWAS/results/phecode_626.12_chr11.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3320:3329/GWAS/results/phecode_626.12_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3320:3329/GWAS/results/phecode_626.12_chr19.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4160:4169/GWAS/results/phecode_751.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4550:4559/GWAS/results/phecode_960.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_747.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_742.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_751.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_960.0_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_743.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_747.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4070:4079/GWAS/results/phecode_742.8_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4100:4109/GWAS/results/phecode_747.11_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4100:4109/GWAS/results/phecode_747.1_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4080:4089/GWAS/results/phecode_743.2_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/R69_chr12.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_276.14_f_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/890:899/GWAS/results/phecode_276.14_f_chr17.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3520:3529/GWAS/results/phecode_671.0_f_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_671.0_f_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_741.2_f_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_681.2_m_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_741.2_m_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4050:4059/GWAS/results/phecode_741.2_f_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4050:4059/GWAS/results/phecode_741.2_m_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/3540:3549/GWAS/results/phecode_681.2_m_chr2.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_747.11_f_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_747.11_m_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_742.8_m_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4070:4079/GWAS/results/phecode_742.8_m_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4100:4109/GWAS/results/phecode_747.11_f_chr1.PHENO1.glm.logistic",
"/cs/labs/michall/roeizucker/phewas_run/runs1/4100:4109/GWAS/results/phecode_747.11_m_chr1.PHENO1.glm.logistic",]

In [ ]:
desc_file = "/cs/labs/michall/roeizucker/phewas_run/description12.csv"
df = pd.read_csv(desc_file)
df["name"] = df["name"]
phen_dic = df.set_index("name").to_dict()["dataset_path"]

In [ ]:
phen_dic

In [ ]:
values = []
for i,file in enumerate(FAILED_FILES):
    chrom = re.findall("chr\d+",file)[0].replace("chr","")
    phen_temp = re.findall("results/.+chr",file)
    if len(phen_temp) == 0:
        print("AAA",file)
        continue
    phenotype = phen_temp[0].replace("_chr","").replace("results/","")
#     print(chrom,phenotype)
    phen_key = phenotype
    if phenotype[0] == "p":
        phen_key = "P" + phen_key[1:].replace("_m","_M").replace("_f","_F")
    gwas_path = phen_dic[phen_key].replace("ukbb_dataset.csv","") + "GWAS"
    base_path = gwas_path
    values.append([i,os.path.join(base_path, "phenotypes/"+ phenotype.lower() + ".txt"),
               os.path.join(base_path, f"results/{phenotype.lower()}_chr{chrom}"),
               f"/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{chrom}"
               ,os.path.join(base_path ,"covariates.txt"),chrom])

#     break


In [ ]:
data_file_path = "/sci/nosnap/michall/roeizucker/emergancy_PWAS_HUB/data_file.csv"
df_data = pd.DataFrame(values, columns=["unsorted_counter","phenotype_path","output_path","partial_chromosome_file_path", "covariates","chr"])
df_data
df_data.to_csv(data_file_path)

In [ ]:
gwas_path = "/sci/nosnap/michall/roeizucker/emergancy_PWAS_HUB/"

mediator_script_path = os.path.join(gwas_path , "mediator.py")
master_script_path = os.path.join(gwas_path , "master.sh")
write_mediator_script(mediator_script_path,data_file_path,gwas_path)
write_master_script(master_script_path,mediator_script_path,gwas_path)

# TODO: add output folder
print(f'''how to run:
sbatch --array=0-263 --mem={MEMORY_AMOUNT + 1}g -c{NUMBER_OF_THREADS} --time=5-0 --requeue --killable --wrap="{master_script_path} 0"''')